# Human_in_the_loop
 - 모델이 스스로 실행하는 것을 신뢰할 수 없는 특정 도구들이 있습니다. 이러한 상황에서 할 수 있는 한 가지 방법은 도구를 사용하기 전에 인간의 승인을 요구하는 것입니다.


In [3]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# from langchain_anthropic import ChatAnthropic

# llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

In [12]:
from operator import itemgetter
from typing import Dict, List

from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool


@tool
def count_emails(last_n_days: int) -> int:
    """Multiply two integers together."""
    return last_n_days * 2

@tool
def send_email(message: str, recipient: str) -> str:
    "Add two integers."
    return f"Successfully sent email to {recipient}."

tools = [count_emails, send_email]
llm_with_tools = llm.bind_tools(tools)

def call_tools(msg: AIMessage) -> List[Dict]:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    #msg.tool_calls =  tool_calls=[{'name': 'count_emails', 'args': {'last_n_days': 5}, 'id': 'call_N4K9PSmOlipu0IQ3CF9TE9WX'}]
    for tool_call in tool_calls:
        #'output': 10} -> 이게 tool_calls에 추가됨
        # tool_calls가 하나라면 굳이 아래 방식 필요 없지만 여럿일 경우 tool_call['name']을 통해 각각 저장해주어야함
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

chain = llm_with_tools | call_tools
chain.invoke("how many emails did i get in the last 5 days?")

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_vSalZiuJbmGsCVjQDrIb6U7P',
  'output': 10}]

# 위 chain에 사용자 승인 더하기

In [14]:
import json

def human_approval(msg: AIMessage) -> Runnable:
    tool_strs = "\n\n".join(
        json.dumps(tool_call, indent=2) for tool_call in msg.tool_calls
    )
    input_msg = (
        f"Do you approve of the following tool invocations\n\n{tool_strs}\n\n"
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no."
    )
    resp = input(input_msg)
    if resp.lower() not in ("yes", "y"):
        raise ValueError(f"Tool invocations not approved:\n\n{tool_strs}")
    return msg


In [15]:
chain = llm_with_tools | human_approval | call_tools

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_8lK5ZvZRjT7ZjPGpMw18kNcN',
  'output': 10}]

In [17]:
chain.invoke("how many emails did i get in the last 5 days?")

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_HgUD6u6OQE7Gl9l5cXnIrURZ',
  'output': 10}]